# Machine Learning : Country access to electricity

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [18]:
df = pd.read_csv('API_EN.GHG.CO2.MT.CE.AR5_DS2_en_csv_v2_6117.csv', sep = ';')
df.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60.0,652230.0,33.93911,67.709953


Predict *future CO₂ emissions (Value_co2_emissions_kt_by_country)* and *Renewable energy share in total final energy consumption (%)* for each country

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Entity                                                            3649 non-null   object 
 1   Year                                                              3649 non-null   int64  
 2   Access to electricity (% of population)                           3639 non-null   float64
 3   Access to clean fuels for cooking                                 3480 non-null   float64
 4   Renewable-electricity-generating-capacity-per-capita              2718 non-null   float64
 5   Financial flows to developing countries (US $)                    1560 non-null   float64
 6   Renewable energy share in the total final energy consumption (%)  3455 non-null   float64
 7   Electricity from fossil fuels (TW

## 2. Dataframe cleaning

### A. Volume of data 

As we're going to do time series prediction we need first to be sure there is enough data, year in our case, to train the model.

In [20]:
df.groupby("Entity")["Year"].count().sort_values()

Entity
French Guiana     1
South Sudan       8
Montenegro       14
Serbia           14
Afghanistan      21
                 ..
Georgia          21
Germany          21
Ghana            21
Ethiopia         21
Zimbabwe         21
Name: Year, Length: 176, dtype: int64

We can see that only 4 country don't have the twenty year background. French Guiana obviously doesn't have enough volume. We take out South Sudan too. We choose to keep country over 14 years data background. 

In [21]:
df = df[~df['Entity'].isin(['French Guiana', 'South Sudan'])]
df.groupby("Entity")["Year"].count().sort_values()

Entity
Montenegro       14
Serbia           14
Afghanistan      21
New Caledonia    21
New Zealand      21
                 ..
Georgia          21
Germany          21
Ghana            21
Ethiopia         21
Zimbabwe         21
Name: Year, Length: 174, dtype: int64

### 2. NAN values

In [22]:
df_nan = df.isna().sum() / len(df) * 100
df_nan.sort_values()

Entity                                                               0.000000
Land Area(Km2)                                                       0.000000
Density\n(P/Km2)                                                     0.000000
Primary energy consumption per capita (kWh/person)                   0.000000
Latitude                                                             0.000000
Longitude                                                            0.000000
Year                                                                 0.000000
Access to electricity (% of population)                              0.247253
Electricity from fossil fuels (TWh)                                  0.576923
Electricity from renewables (TWh)                                    0.576923
Low-carbon electricity (% electricity)                               1.153846
Electricity from nuclear (TWh)                                       3.461538
Access to clean fuels for cooking                               

The columns Renewables (% equivalent primary energy) and Financial flows to developing countries have more than half of there data missing so we drop them

In [23]:
df = df.drop(columns = ['Renewables (% equivalent primary energy)', 'Financial flows to developing countries (US $)' ])
df

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,60.0,652230.0,33.939110,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,60.0,652230.0,33.939110,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,179.426579,60.0,652230.0,33.939110,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,8.832278,190.683814,60.0,652230.0,33.939110,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,1.414118,211.382074,60.0,652230.0,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,Zimbabwe,2016,42.561730,29.8,62.88,81.90,3.50,0.0,3.32,48.680350,3227.68020,10.00,11020.000460,0.755869,1464.588957,38.0,390757.0,-19.015438,29.154857
3645,Zimbabwe,2017,44.178635,29.8,62.33,82.46,3.05,0.0,4.30,58.503407,3068.01150,9.51,10340.000150,4.709492,1235.189032,38.0,390757.0,-19.015438,29.154857
3646,Zimbabwe,2018,45.572647,29.9,82.53,80.23,3.73,0.0,5.46,59.412407,3441.98580,9.83,12380.000110,4.824211,1254.642265,38.0,390757.0,-19.015438,29.154857
3647,Zimbabwe,2019,46.781475,30.1,81.40,81.50,3.66,0.0,4.58,55.582527,3003.65530,10.47,11760.000230,-6.144236,1316.740657,38.0,390757.0,-19.015438,29.154857


In [24]:
nan_cols = [
    'Access to electricity (% of population)',
    'Electricity from fossil fuels (TWh)',
    'Electricity from renewables (TWh)',
    'Low-carbon electricity (% electricity)',
    'Electricity from nuclear (TWh)',
    'Access to clean fuels for cooking',
    'Renewable energy share in the total final energy consumption (%)',
    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
    'gdp_per_capita',
    'gdp_growth',
    'Value_co2_emissions_kt_by_country',
    'Renewable-electricity-generating-capacity-per-capita'
]


To manage the rest of the missing data, we take into account both dimensions: time and variables.
If a missing value for a variable corresponds to an isolated year for a country, it is replaced using a moving average based on the previous and the following year.
However, if a variable is entirely missing for a country (there are no observations for any year), we impute it with the average value of that variable across all countries

In [ ]:
df = df.sort_values(['Entity', 'Year'])


In [12]:

# Étape 1 : moyenne mobile pour les valeurs manquantes isolées
for col in nan_cols:
    df[col] = df.groupby('Entity')[col].transform(
        lambda x: x.fillna(x.rolling(window=3, min_periods=1).mean())
    )

# Étape 2 : si une variable est totalement manquante pour un pays,
# on la remplace par la moyenne globale de cette variable
for col in nan_cols:
    df[col] = df.groupby('Entity')[col].transform(
        lambda x: x.fillna(df[col].mean())
    )


In [13]:
df_nan = df.isna().sum() / len(df) * 100
df_nan.sort_values()

Entity                                                              0.0
Land Area(Km2)                                                      0.0
Density\n(P/Km2)                                                    0.0
gdp_per_capita                                                      0.0
gdp_growth                                                          0.0
Value_co2_emissions_kt_by_country                                   0.0
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         0.0
Primary energy consumption per capita (kWh/person)                  0.0
Latitude                                                            0.0
Low-carbon electricity (% electricity)                              0.0
Electricity from nuclear (TWh)                                      0.0
Electricity from fossil fuels (TWh)                                 0.0
Renewable energy share in the total final energy consumption (%)    0.0
Renewable-electricity-generating-capacity-per-capita            